# 1. Loading data

In [ ]:
import os, sys
sys.path.append(os.getcwd())

import numpy as np
import pandas as pd

In [ ]:
# Load data
sigma70 = pd.read_csv("20211213.Sigma70.txt", sep = '\t', names = ["name", "seq", "strand", "express"])

# Create label
label = []
for i in range(496):
    label.append(0)
for i in range(10):
    label.append(1)
for i in range(495):
    label.append(0)
    
sigma70["labels"] = [label] * len(sigma70)

# Split independent testing dataset
ind_sigma70 = sigma70.sample(n=200, random_state=37)
train_sigma70 = sigma70[~sigma70.index.isin(ind_sigma70.index)]

val_sigma70 = ind_sigma70.sample(n=100, random_state=11)
test_sigma70 = ind_sigma70[~ind_sigma70.index.isin(val_sigma70.index)]

# 2. Import libraries and define functions

## Import libraries

In [ ]:
import os, sys
sys.path.append(os.getcwd())

import math
import time
import json
import random
import itertools

import torch
import torch.nn as nn
from torch.nn import init
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd

from tqdm import tqdm
from torch.autograd import Variable
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold

from torchsummary import summary

torch.manual_seed(1)
use_cuda = torch.cuda.is_available()
if use_cuda:
    gpu = 0
    
BATCH_SIZE = 256 # Batch size


## Define function used in the training process

In [ ]:
# Generating sliding sequences from the data

def sliding(df, random_sampling=False):
    negative = []
    seqs = []
    labels = []

    for idx, row in df.iterrows():
        # Negative data
        for i in range(0, 371): # 371 sequences
            e = i + 150
            negative.append(row["seq"][i:e])

        for i in range(431, 851): # 420 sequences
            e = i + 150
            negative.append(row["seq"][i:e])
        
        if random_sampling == True:
            # Random sampling the negative data to 60 sequences
            random.seed(idx)
            random_negative_seq = random.sample(negative, 60)
            for negative_seq in random_negative_seq:
                seqs.append(negative_seq)

            for i in range(len(random_negative_seq)):
                labels.append(label[0:150])
        elif random_sampling == False:
            random.seed(idx)
            random_negative_seq = random.sample(negative, 400)
            for negative_seq in random_negative_seq:
                seqs.append(negative_seq)

            for i in range(len(random_negative_seq)):
                labels.append(label[0:150])

        # Positive data
        for i in range(371, 431): # 60 sequences
            e = i + 150
            seqs.append(row["seq"][i:e])
            labels.append(row["labels"][i:e])

    labels_com = []
    for lab in labels:
        lab = [lab]
        lab_arr = np.asarray(lab)
        lab_arr = lab_arr.transpose(1,0)
        labels_com.append(lab_arr)

    return seqs, np.asarray(labels_com)

In [ ]:
# Reference for stacked energy and bendability values

energy_ref = {'GC': -14.59, 'GT': -10.51, 'AC': -10.51, 'GA': -9.81, 'TC': -9.81, 'CG': -9.69,
       'CC': -8.26, 'GG': -8.26, 'AT': -6.57, 'CA': -6.57, 'TG': -6.57, 'CT': -6.78,
       'AG': -6.78, 'TT': -5.37, 'AA': -5.37, 'TA': -3.82}
energy = -np.array(list(energy_ref.values()))
energy_normed = 2*(energy - np.min(energy))/(np.max(energy) - np.min(energy)) - 1
energy_ref_normed = dict(zip(energy_ref.keys(), energy_normed))

bendability_ref = {'AAT': -0.28, 'AAA': -0.274, 'CCA': -0.246, 'AAC': -0.205, 'ACT': -0.183, 'CCG': -0.136,
    'ATC': -0.11, 'AAG': -0.081, 'CGC': -0.077, 'AGG': -0.057, 'GAA': -0.037, 'ACG': -0.033,
    'ACC': -0.032, 'GAC': -0.013, 'CCC': -0.012, 'ACA': -0.006, 'CGA': -0.003, 'GGA': 0.013,
    'CAA': 0.015, 'AGC': 0.017, 'GTA': 0.025, 'AGA': 0.027, 'CTC': 0.031, 'CAC': 0.04,
    'TAA': 0.068, 'GCA': 0.076, 'CTA': 0.09, 'GCC': 0.107, 'ATG': 0.134, 'CAG': 0.175,
    'ATA': 0.182, 'TCA': 0.194, 'ATT': -0.28, 'TTT': -0.274, 'TGG': -0.246, 'GTT': -0.205,
    'AGT': -0.183, 'CGG': -0.136, 'GAT': -0.11, 'CTT': -0.081, 'GCG': -0.077, 'CCT': -0.057,
    'TTC': -0.037, 'CGT': -0.033, 'GGT': -0.032, 'GTC': -0.013, 'GGG': -0.012, 'TGT': -0.006,
    'TCG': -0.003, 'TCC': 0.013, 'TTG': 0.015, 'GCT': 0.017, 'TAC': 0.025, 'TCT': 0.027,
    'GAG': 0.031, 'GTG': 0.04, 'TTA': 0.068, 'TGC': 0.076, 'TAG': 0.09, 'GGC': 0.107,
    'CAT': 0.134, 'CTG': 0.175, 'TAT': 0.182, 'TGA': 0.194}

bendability = np.array(list(bendability_ref.values()))
bendability_normed = 2*(bendability - np.min(bendability))/(np.max(bendability) - np.min(bendability)) - 1
bendability_ref_normed = dict(zip(bendability_ref.keys(), bendability_normed))

In [ ]:
# Dataset for pytorch

class SeqDataset(Dataset):
    def __init__(self, seqs, labels, bend_ref):
        self.seqs = seqs
        self.labels = labels
        
        self.bend_ref = bend_ref
        self.avg_bend = sum(bend_ref.values())/len(bend_ref)
        
        assert len(self.labels) == len(self.seqs)
    
    def seq2onehot(self, seq):   
        module = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
        promoter_onehot = []
        for item in seq:
            if item == 't' or item == 'T':
                promoter_onehot.append(module[0])
            elif item == 'a' or item == 'A':
                promoter_onehot.append(module[1])
            elif item == 'g' or item == 'G':
                promoter_onehot.append(module[2])
            elif item == 'c' or item == 'C':
                promoter_onehot.append(module[3])
            else:
                promoter_onehot.append([0,0,0,0])

        data = np.array(promoter_onehot)
        data = np.float32(data)
        data = np.transpose(data, (1,0))

        return data
    
    def energyStacking(self, seq):
        energy = []
        energy.append(self.avg_energy)
        
        for i in range(len(seq) - 1):
            dimer = ''.join(seq[i:i+2])
            dimer_val = self.energy_ref[dimer]
            #energy.append(energy[-1] + dimer_val)
            energy.append(dimer_val)
        
        return np.float32(np.array(energy))
    
    def dnaBendability(self, seq):
        bend = []
        bend.append(self.avg_bend)
        
        for i in range(len(seq) - 2):
            trimer = ''.join(seq[i:i+3])
            trimer_bend = self.bend_ref[trimer]
            bend.append(trimer_bend)
            
        bend.append(self.avg_bend)
        
        return np.float32(np.array(bend))

    def __len__(self):
        return len(self.seqs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        seq = self.seqs[idx]
        seq = list(itertools.chain.from_iterable(seq))
        onehot = self.seq2onehot(seq)
        bend = self.dnaBendability(seq)
        seq = np.vstack([onehot, bend])
        
        label = self.labels[idx]
        label = np.float32(label)
        label = np.transpose(label, (1,0))
        label = torch.tensor(label)

        return seq, label

In [ ]:
def evaluation_metrics(tp, fp, tn, fn):
    acc = (tp+tn)/(tp+fp+tn+fn)
    recall = tp/(tp+fn)
    spec = tn/(tn+fp)
    f1 = 2*tp/(2*tp+fp+fn)
    mcc = (tp*tn-fp*fn)/math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    return acc, spec, recall, f1, mcc

def evaluate_model(model, dataset, dataloader, threshold=0.05):
    recon_data = []
    recon_conf_data = []
    labels = []
    with torch.no_grad():    
        for i, (x, x_label) in enumerate(dataloader):
            x = x.cuda()
            recon, recon_conf = model(x)
            if i == 0:
                recon_data = recon.cpu().numpy()
            else:
                recon_data = np.vstack((recon_data, recon.cpu().numpy()))
    
    for i in range(len(recon_data)):
        recon_data[i][recon_data[i] >= threshold] = 1
        recon_data[i][recon_data[i] < threshold] = 0
    
    pos_list = []
    neg_list = []
    correct_neg = 0
    correct_pos = 0

    for i in range(len(dataset)):
        if sum(dataset[i][1][0].detach().numpy()) == 0.0:
            neg_list.append(i)
            if sum(recon_data[i][0]) == 0.0:
                correct_neg += 1
        else:
            pos_list.append(i)
            if sum(recon_data[i][0]) != 0.0:
                correct_pos += 1
                
    fn = len(pos_list) - correct_pos
    fp = len(neg_list) - correct_neg
    
    return evaluation_metrics(correct_pos, fp, correct_neg, fn)

In [ ]:
# Focal loss used in the training process

class FocalLoss(nn.Module):

    def __init__(self, alpha=.2, gamma=3., reduction="mean"):
        super(FocalLoss, self).__init__()
#         self.alpha = torch.tensor([alpha, 1-alpha]).cuda()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, recon_x, x):
        # Calculate BCE loss
        BCE_loss = F.binary_cross_entropy(recon_x, x)
        
        # Calculate focal loss
        pt = torch.exp(-BCE_loss)
        # p = torch.sigmoid(recon_x)
        # pt = p * x + (1 - p) * (1 - x)
        # pt = recon_x * x + (1 - recon_x) * (1 - x)
        F_loss = (1-pt)**self.gamma * BCE_loss
    
        if self.alpha > 0:
            at = self.alpha * x + (1 - self.alpha) * (1 - x)
            F_loss = at * F_loss
        
        if self.reduction == "mean":
            F_loss = F_loss.mean()
        elif self.reduction == "sum":
            F_loss = F_loss.sum()
        
        return F_loss

criterion = FocalLoss()

class_loss = nn.CrossEntropyLoss()

In [ ]:
# Define the model

class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.squeeze = nn.AdaptiveAvgPool1d(1)
        self.excitation = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.squeeze(x).view(b, c)
        y = self.excitation(y).view(b, c, 1)
        return x * y.expand_as(x)
    

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None, seq_length=150):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels*4
        
        self.doubleConv = nn.Sequential(
            nn.Conv1d(in_channels, in_channels, kernel_size=7, padding=3, groups=in_channels), # The sequence length will remain the same
            nn.LayerNorm(normalized_shape=[in_channels, seq_length]),
            nn.Conv1d(in_channels, mid_channels, kernel_size=7, padding=3),
            nn.Conv1d(mid_channels, out_channels, kernel_size=7, padding=3), # The sequence length will remain the same
            nn.GELU()
        )
        
    def forward(self, x):
        return self.doubleConv(x)

    
class Down(nn.Module):
    """Downsampling using maxpooling, followed by 2 layers of convolution"""
    def __init__(self, in_channels, out_channels, mid_channels=None, special=False, seq_length=150):
        super().__init__()
        if not special:
            self.maxpoolConv = nn.Sequential(
                nn.MaxPool1d(2),
                DoubleConv(in_channels, out_channels, mid_channels, seq_length)
            )
        else:
            self.maxpoolConv = nn.Sequential(
                nn.MaxPool1d(kernel_size=2, stride=2, padding=1), 
                DoubleConv(in_channels, out_channels, mid_channels, seq_length)
            )
        
    def forward(self, x):
        return self.maxpoolConv(x)


class Up(nn.Module):
    """Upsampling (with skip connection) then 2 conv layers"""
    def __init__(self, in_channels, out_channels, mid_channels=None, special=False, seq_length=150):
        super().__init__()
        
        if not special:
            self.up = nn.ConvTranspose1d(in_channels, in_channels // 2, kernel_size=2, stride=2) 
            self.conv = DoubleConv(in_channels, out_channels, mid_channels=mid_channels, seq_length=seq_length)
        else:
            self.up = nn.ConvTranspose1d(in_channels, in_channels // 2, kernel_size=3, stride=2, padding=1) #[256, 38] -> [128, 75]
            self.conv = DoubleConv(in_channels, out_channels, mid_channels=mid_channels, seq_length=seq_length) #[256, 38] -> [128, 75]
            
    def forward(self, x1, x2):
        x1 = self.up(x1) # [128, 75]
        x = torch.cat([x2, x1], dim=1) # [256, 75]
        return self.conv(x)
        

class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)
    

class MLP(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(in_dim, 64)
        self.dropout1 = nn.Dropout()
        self.relu1 = nn.GELU()
        self.linear2 = nn.Linear(64, out_dim)
        self.dropout2 = nn.Dropout()
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.dropout1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        return x
    
    
class Unet1D(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(Unet1D, self).__init__()
                
        self.inConv = DoubleConv(n_channels, 32) # (150+2*1-3)/1+1 = 150 (4 -> 32)
        self.selayer1 = SELayer(32)
        self.down1 = Down(32, 64, seq_length=75) # (Maxpooling -> 75) (32 -> 64)
        self.selayer2 = SELayer(64)
        self.down2 = Down(64, 128, special=True, seq_length=38) # (Maxpooling -> 38) (64 -> 128)
        self.selayer3 = SELayer(128)
        self.down3 = Down(128, 256, seq_length=19) # (Maxpooling -> 19) (128 -> 256)
        
        self.up1 = Up(256, 128, mid_channels=512, seq_length=38) # (ConvTranspose -> 38) (256 -> 128)
        self.up2 = Up(128, 64, mid_channels=256, special=True, seq_length=75) # (ConvTranspose -> 75) (128 -> 64)
        self.up3 = Up(64, 32, mid_channels=128) # (ConvTranspose -> 150) (64 -> 32)
        self.outConv = OutConv(32, n_classes)
        self.classConv = MLP(150, 1)
        
    def forward(self, x):
        x1 = self.inConv(x) # [x, 32, 150]
        x1 = self.selayer1(x1)
        x2 = self.down1(x1) # [x, 64, 75]
        x2 = self.selayer2(x2)
        x3 = self.down2(x2) # [x, 128, 38]
        x3 = self.selayer3(x3)
        x4 = self.down3(x3) # [x, 256, 19]
        
        x = self.up1(x4, x3) # [x, 128, 38]
        x = self.up2(x, x2) # [x, 64, 75]
        x = self.up3(x, x1) # [x, 32, 150]
        out = self.outConv(x) # [x, 1, 150]
        #out = torch.sigmoid(out)
        #out_conf = self.classConv(out.view(out.size()[0], -1))
        out_conf = self.classConv(out.squeeze(1))
        return (torch.sigmoid(out), torch.sigmoid(out_conf))

# 3. Training model

In [ ]:
# Generate sliding sequences
train_seqs, train_labels = sliding(train_sigma70)
val_seqs, val_labels = sliding(val_sigma70)

In [ ]:
# Create dataset
train_set = SeqDataset(train_seqs, train_labels, bendability_ref_normed)
val_set = SeqDataset(val_seqs, val_labels, bendability_ref_normed)

# Create dataloader
train_dl = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# Define model
unet = Unet1D(5, 1)
unet.cuda()
optimizer = torch.optim.AdamW(unet.parameters(), lr=0.001)

train_loss_his = []
val_loss_his = []

for epoch in range(1000):
    print("Epoch {}".format(epoch))
    start = time.time()
    train_eloss = 0
    val_eloss = 0

    unet.train()
    for x, x_label in train_dl:
        x = x.cuda()
        x_label = x_label.cuda()
        recon, recon_conf = unet(x)
        
        focal_loss = criterion(recon, x_label)
        
        conf = class_loss(recon_conf, torch.sum(x_label, dim = -1))
        loss = focal_loss + 0.1 * conf

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_eloss += loss.cpu().item()

    unet.eval()
    with torch.no_grad():
        for i, (x, x_label) in enumerate(val_dl):
            x = x.cuda()
            x_label = x_label.cuda()
            recon, recon_conf = unet(x)
            val_focal_loss = criterion(recon, x_label) 
            val_conf = class_loss(recon_conf, torch.sum(x_label, dim = -1))
            val_loss = val_focal_loss + 0.1 * val_conf

            val_eloss += val_loss.cpu().item()

    train_loss_his.append(train_eloss/len(train_dl))
    val_loss_his.append(val_eloss/len(val_dl))

    if epoch == 0:
        loss_val_history = val_loss_his[-1]
        patience = 0
    else:
        loss_val_history = np.append(loss_val_history, val_loss_his[-1])

    if val_loss_his[-1] < 0.00000000000000001 + np.min(loss_val_history):
        patience = 0
        model = "best_model_BEND_K7.pt"
        torch.save(unet.state_dict(), model)
    else:
        patience +=1

    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    print(epoch, patience, val_eloss/len(val_dl), np.min(loss_val_history))

    if patience == 5:
        model = "best_model_BEND_K7.pt"
        unet.load_state_dict(torch.load(model))

        break
